In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import f_classif 
from sklearn.feature_selection import RFE 
from sklearn.ensemble import ExtraTreesClassifier  

In [ ]:
train = pd.read_csv("datasets/training.csv") #filepath
train

In [ ]:
subset_tau =train[["LifeTime", "FlightDistance", "dira", "mass" , "pt" , "IP", "IPSig" , "VertexChi2","signal" ]]
subset_mu = train[["p0_p", "p1_p", "p2_p", "p0_pt" , "p1_pt" , "p2_pt", "p0_eta" , "p1_eta","p2_eta","signal"  ]]
subset_detector = train.drop(["LifeTime", "FlightDistance", "dira", "mass" , "pt" , "IP", "IPSig" , "VertexChi2","signal", "p0_p", "p1_p", "p2_p", "p0_pt" , "p1_pt" , "p2_pt", "p0_eta" , "p1_eta","p2_eta" ], axis=1)

In [ ]:
# Variable numérica
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 5))
axes = axes.flat
columnas = subset_tau.select_dtypes(include=['float64', 'int']).columns
columnas = columnas.drop('signal') # objetivo

for i, colum in enumerate(columnas):
    sns.regplot(
        x           = subset_tau[colum],
        y           = subset_tau['signal'],
        color       = "gray",
        marker      = '.',
        scatter_kws = {"alpha":0.4},
        line_kws    = {"color":"r","alpha":0.7},
        ax          = axes[i]
    )
    axes[i].set_title(f"Señal/Background vs {colum}", fontsize = 7, fontweight = "bold")
    axes[i].tick_params(labelsize = 6)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

# Se eliminan los axes vacíos
for i in [8]:
    fig.delaxes(axes[i])
    
fig.tight_layout()
plt.subplots_adjust(top=0.9)
fig.suptitle('Correlación con Señal/background', fontsize = 10, fontweight = "bold");

In [ ]:
subset_tau.isnull().any().any()

## Selección de datos

1. Objetivo: y
2. Características: x

En este caso queremos separar la señal del background

In [ ]:
x = subset_tau['signal']
k = 2  # número de atributos a seleccionar
entrenar = subset_tau.drop(['signal'], axis=1)
columnas = list(entrenar.columns.values)
seleccionadas = SelectKBest(f_classif, k=k).fit(entrenar, x)
atrib = seleccionadas.get_support()
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

In [ ]:
modelo = ExtraTreesClassifier() 
erec = RFE(modelo)  
erec = erec.fit(entrenar, x)

In [ ]:
atrib = erec.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
atributos

In [ ]:
# Importancia de atributos.
modelo.fit(entrenar, x)
modelo.feature_importances_[:4]